In [ ]:
# notebooks/data_analysis.ipynb - Initial content
# You can create this as a .ipynb file

"""
SoorGreen Waste Data Analysis Notebook

This notebook provides interactive analysis of waste collection data.
"""

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Database connection
from config.config import config

# Setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Connect to database
engine = create_engine(config.DB_CONNECTION_STRING)

# Load data
print("Loading waste reports data...")
query = """
SELECT 
    w.ReportId,
    w.WasteTypeId,
    wt.Name as WasteType,
    w.EstimatedKg,
    w.CreatedAt,
    w.Lat,
    w.Lng,
    m.Name as Municipality,
    u.FullName as CitizenName
FROM WasteReports w
JOIN WasteTypes wt ON w.WasteTypeId = wt.WasteTypeId
LEFT JOIN Municipalities m ON w.MunicipalityId = m.MunicipalityId
LEFT JOIN Users u ON w.UserId = u.UserId
WHERE w.CreatedAt >= DATEADD(month, -3, GETDATE())
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} records")

# Basic statistics
print("\n=== Basic Statistics ===")
print(f"Total waste reported: {df['EstimatedKg'].sum():.2f} kg")
print(f"Average per report: {df['EstimatedKg'].mean():.2f} kg")
print(f"Number of unique citizens: {df['CitizenName'].nunique()}")
print(f"Number of municipalities: {df['Municipality'].nunique()}")

# Waste composition
waste_composition = df.groupby('WasteType')['EstimatedKg'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
waste_composition.plot(kind='bar', color='skyblue')
plt.title('Total Waste by Type (Last 3 Months)')
plt.xlabel('Waste Type')
plt.ylabel('Total Weight (kg)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Time series analysis
df['Date'] = pd.to_datetime(df['CreatedAt']).dt.date
daily_volume = df.groupby('Date')['EstimatedKg'].sum()

plt.figure(figsize=(12, 6))
plt.plot(daily_volume.index, daily_volume.values, marker='o', linestyle='-', color='green')
plt.title('Daily Waste Volume Trend')
plt.xlabel('Date')
plt.ylabel('Weight (kg)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Geographical distribution
if not df[['Lat', 'Lng']].isnull().all().all():
    plt.figure(figsize=(10, 8))
    plt.scatter(df['Lng'], df['Lat'], alpha=0.6, c='red', s=df['EstimatedKg']*10)
    plt.title('Waste Report Locations (Size = Weight)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("\nAnalysis complete! Use this notebook for further exploration.")